## Sri Nandini Ganisetty
#### A report on community chat system extension to add new functionality to existing chat system that currently supports 75 million users. The users can include attached file with their messages. This proposed system keeps track userid, date, filetype and the file itself.

#### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [9]:
session.execute("DROP KEYSPACE IF EXISTS midtermpython")

In [11]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04


In [12]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS midtermpython WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [31]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

midtermpython
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04


Below we create a user table contains information such as user_id, date, fileType and file itself for the chat system extension. user_id is made the primary key

In [15]:
session.execute("""
CREATE TABLE IF NOT EXISTS midtermpython.user ( 
    user_id INT, 
    date TEXT, 
    fileType TEXT, 
    file TEXT, 
    PRIMARY KEY(user_id)
);
""")

### Load data: Below we Load the data by reading it from csv file.

In [16]:
df = pd.read_csv('data.csv')
df

,user_id,date,fileType,file
0,1000,'10042023','Image','Test'
1,1001,'10012023','Image','Test'
2,1002,'10022023','video','Test'
3,1003,'10032022','video','Test'
4,1004,'01042023','video','Test'
5,1005,'11042021','Image','Test'
6,1006,'19042021','Image','Test'
7,1007,'10042017','file','Test'
8,1008,'10062023','file','Test'
9,1009,'10072023','file','Test'


#### Below we can view all the records that are inserted into our database

In [17]:
for index, row in df.iterrows():
    print(f"user_id = {row.user_id}, date ={row.date}, fileType = {row.fileType}, file = {row.file}")

user_id = 1000, date ='10042023', fileType = 'Image', file =  'Test'
user_id = 1001, date ='10012023', fileType = 'Image', file = 'Test'
user_id = 1002, date ='10022023', fileType = 'video', file = 'Test'
user_id = 1003, date ='10032022', fileType = 'video', file = 'Test'
user_id = 1004, date ='01042023', fileType = 'video', file = 'Test'
user_id = 1005, date ='11042021', fileType = 'Image', file = 'Test'
user_id = 1006, date ='19042021', fileType = 'Image', file = 'Test'
user_id = 1007, date ='10042017', fileType = 'file', file = 'Test'
user_id = 1008, date ='10062023', fileType = 'file', file = 'Test'
user_id = 1009, date ='10072023', fileType = 'file', file = 'Test'
user_id = 1010, date ='10112023', fileType = 'file', file = 'Test'
user_id = 1011, date ='10032023', fileType = 'Image', file = 'Test'


In [18]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES ({row.user_id}, {row.date}, {row.fileType}, {row.file});
        """
    )
    session.execute(f"""
        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES ({row.user_id}, {row.date}, {row.fileType}, {row.file});
        """
    )


        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1000, '10042023', 'Image',  'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1001, '10012023', 'Image', 'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1002, '10022023', 'video', 'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1003, '10032022', 'video', 'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1004, '01042023', 'video', 'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1005, '11042021', 'Image', 'Test');
        

        INSERT INTO midtermpython.user (user_id, date, fileType, file)     
        VALUES (1006, '19042021', 'Image', 'Test');
        

        INSERT INTO midtermpytho

In [19]:
rows = session.execute("select (user_id, date, fileType, file) from midtermpython.user")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")

user_id=1009, date=10072023, fileType=file, file=Test
user_id=1006, date=19042021, fileType=Image, file=Test
user_id=1008, date=10062023, fileType=file, file=Test
user_id=1011, date=10032023, fileType=Image, file=Test
user_id=1004, date=01042023, fileType=video, file=Test
user_id=1007, date=10042017, fileType=file, file=Test
user_id=1005, date=11042021, fileType=Image, file=Test
user_id=1001, date=10012023, fileType=Image, file=Test
user_id=1003, date=10032022, fileType=video, file=Test
user_id=1002, date=10022023, fileType=video, file=Test
user_id=1000, date=10042023, fileType=Image, file=Test
user_id=1010, date=10112023, fileType=file, file=Test


store_file functionality : Below is the functionality to store any incoming file and it's information. We can do the follow action by inserting the incoming record.
After insertion, the inserted column is printed to view below

In [28]:
#insert one record
session.execute(f"""
        INSERT INTO midtermpython.user (user_id, date, fileType, file)    
        VALUES (2000,'09092023', 'Image', 'file');
        """
)
#view inserted record
rows = session.execute("select (user_id, date, fileType, file) from midtermpython.user WHERE user_id = 2000")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")

rows = session.execute("select (user_id, date, fileType, file) from midtermpython.user")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")

user_id=2000, date=09092023, fileType=Image, file=file
user_id=1009, date=10072023, fileType=file, file=Test
user_id=1006, date=19042021, fileType=Image, file=Test
user_id=1008, date=10062023, fileType=file, file=Test
user_id=1011, date=10032023, fileType=Image, file=Test
user_id=1004, date=01042023, fileType=video, file=Test
user_id=1007, date=10042017, fileType=file, file=Test
user_id=1005, date=11042021, fileType=Image, file=Test
user_id=1001, date=10012023, fileType=Image, file=Test
user_id=2000, date=09092023, fileType=Image, file=file
user_id=1003, date=10032022, fileType=video, file=Test
user_id=1002, date=10022023, fileType=video, file=Test
user_id=1010, date=10112023, fileType=file, file=Test


#### retrieve_file functionality: Below is the functionality to retrieve theinformation of any file by its user id. We can do the follow action by selecting the record from our user database.

In [29]:
#for example, if we want to search a file with user_id = 2000
user_id_to_search = 2000;
rows = session.execute(f"""select (user_id, date, fileType, file) from midtermpython.user WHERE user_id = {user_id_to_search}""")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")


user_id=2000, date=09092023, fileType=Image, file=file


#### delete_file functionality: Below is the functionality to delete the file information of any file by its user id. We can do the follow action by delete statement the record from our user database.

In [30]:
user_id_to_deleted = 1000;
rows = session.execute(f"""DELETE from midtermpython.user where user_id= {user_id_to_deleted}""")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")
rows = session.execute("select (user_id,date,fileType,file) from midtermpython.user")

#display records after deletion
rows = session.execute("select (user_id, date, fileType, file) from midtermpython.user")
for row in rows:
    print(f"user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")

# for row in rows:
#     print(f"deleted record is user_id={row[0][0]}, date={row[0][1]}, fileType={row[0][2]}, file={row[0][3]}")


user_id=1009, date=10072023, fileType=file, file=Test
user_id=1006, date=19042021, fileType=Image, file=Test
user_id=1008, date=10062023, fileType=file, file=Test
user_id=1011, date=10032023, fileType=Image, file=Test
user_id=1004, date=01042023, fileType=video, file=Test
user_id=1007, date=10042017, fileType=file, file=Test
user_id=1005, date=11042021, fileType=Image, file=Test
user_id=1001, date=10012023, fileType=Image, file=Test
user_id=2000, date=09092023, fileType=Image, file=file
user_id=1003, date=10032022, fileType=video, file=Test
user_id=1002, date=10022023, fileType=video, file=Test
user_id=1010, date=10112023, fileType=file, file=Test
